# Product Propensity Modeling

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

## Data Ingestion + Exploration + Splitting

### Data Ingestion

In [ ]:
product = pd.read_excel('product_sales_v3.xlsx', sheet_name='Data')

### Data Exploration

We perform the below exploratory data analysis to get a basic undersatnding of the data, without letting it leak into our predictions

There are no duplicate rows:

In [ ]:
product[product.duplicated()]

We convert the order_day field to a datetime:

In [ ]:
#convert the order_day to a datetime MM/DD/YYYY format
product.order_day = pd.to_datetime(product.order_day, format='%m/%d/%Y')

We visualize the acceptance rates per segment:

In [ ]:
product.groupby('segment').agg({'accept': 'mean'}).plot.bar(title='acceptance rate per segment')

We visualize the acceptance rate per marketing area:

In [ ]:
product.groupby('dma').agg({'accept': 'mean'}).plot.bar(title='acceptance rate per dma')

We visualize the acceptance rate per year:

In [ ]:
#acceptance trends by year
product[['accept']].groupby([product['order_day'].astype(str).str[:-6]]).mean().plot.bar(
    title='acceptance rate per year')

In [ ]:
product[['order_day', 'accept']].groupby([product['order_day'].astype(str).str[:-6]]).mean()

We visualize the acceptance rate per month:

In [ ]:
#acceptance trends by month
product[['accept']].groupby([product['order_day'].astype(str).str[5:7]]).mean().plot.bar(
    title='acceptance rate per month')

We visualize the acceptance rate over continuous time:

In [ ]:
#acceptance trends by month/year
product[['accept']].groupby([product['order_day'].astype(str).str[:7]]).mean().plot.line(
    title='acceptance rate per month-year')

We visualize the acceptance volume per year:

In [ ]:
product[['accept']].groupby([product['order_day'].astype(str).str[:-6]]).count().plot.bar(
    title='acceptance volume per year')

We visualize the acceptance volume per month:

In [ ]:
product[['accept']].groupby([product['order_day'].astype(str).str[5:7]]).count().plot.bar(
    title='acceptance volume per month')

We visualize the top 10 days with the most acceptances:

In [ ]:
product[['accept']].groupby([product['order_day'].astype(str).str[:10]]).count().sort_values(by='accept',                                                                                    ascending=False).head(
    10).plot.bar(title='acceptance volume - top 10 days')

### Train/Test Split

Time-based splitting

In [ ]:
#create time-based split, with first 75% in train data, last 25% in test data
from sklearn.model_selection import train_test_split

product.sort_values(by='order_day', inplace=True)  #use for time series sort to sort by date before splitting
product_X_data = product.drop('accept', axis=1)  #remove the target variable from the X data
product_y_data = product[['accept']]
product_x_train, product_x_test, product_y_train, product_y_test = train_test_split(product_X_data, product_y_data, test_size=.25, random_state=100, shuffle=False)

Check distribution of acceptance rate for above splitting approach:

In [ ]:
product_y_train['accept'].value_counts(normalize=True)

In [ ]:
product_y_test['accept'].value_counts(normalize=True)

Check distribution of order day for above splitting approach:

In [ ]:
product_x_train['order_day'].sort_values()

In [ ]:
product_x_test['order_day'].sort_values()

## Data Preprocessing Pipeline Creation

### Data Cleaning + Feature Engineering Functions

Convert data types to predefined dictionary of field:dtype

In [ ]:
def convert_dtypes(df, data_type_transformation):
    """convert data types to predefined dictionary of field:dtype"""
    df = df.astype(data_type_transformation)
    return df

Convert Y/NaN field to Y/N - this is meant for the tos_flg field

In [ ]:
def replace_nan(data, col):
    """convert NaN's to N for certain Y/N columns"""
    data[col] = data[col].fillna('N')
    return data

Convert numerical blank field values to 0 - this is intended for the deposit_onhand_amt field

In [ ]:
def replace_blank(data, col):
    """convert blanks to 0 for certain numeric columns"""
    data[col] = data[col].fillna(0)
    return data

Keep the left n characters of a columns - this is intended for the zipcode field

In [ ]:
#get the left 5 digits of zipcode
def left_strip(data, col, num_char):
    """keep only the left n of characters"""
    data[col] = data[col].astype('str').str[:num_char]
    return data

Convert zipcode to latitude/longitude value - this is intended for the zipcode field

In [ ]:
def zip_to_latlong(df, left_col, right_col, mapping_file):
    """convert zipcode to latitude/longitude"""
    zip_code_map = pd.read_csv(mapping_file)
    zip_code_map[right_col] = zip_code_map[right_col].astype('str')
    data_merged = pd.merge(df, zip_code_map, left_on=left_col, right_on=right_col, how='left')
    data_merged = data_merged.drop(left_col, axis=1)
    data_merged = data_merged.drop(right_col, axis=1)
    return data_merged

Convert true/false to 1/0 - this is meant for boolean columns

In [ ]:
def replace_boolean(data):
    """convert True/False to 1/0 for boolean columns"""
    for col in data.select_dtypes(include=[bool]):
        data[col].replace(True, 1, inplace=True)
        data[col].replace(False, 0, inplace=True)
    return data

Convert Y/N columns to 1/0 - this is meant for boolean columns that are represented as Y/N strings

In [ ]:
#convert Y/N columns to 1/0, hardcode columns to avoid picking up true Y/N non-boolean values?
def replace_yn(data):
    """convert Y/N to 1/0 for Y/N columns"""
    for col in data.select_dtypes(include=[object]):
        data[col].replace('Y', 1, inplace=True)
        data[col].replace('N', 0, inplace=True)
    return data

Convert non-numeric values to NaN in a numeric column - this is meant for the term_length column, where there are several term length values that are non-numeric in the dataset:

In [ ]:
def replace_non_numeric(data, col):
    '''convert non-numeric data in a numeric field to NaN'''
    data[col] = data[col].astype('str').replace(r'^[^0-9]', np.nan, regex=True).astype('float')
    return data

Convert order_day to seasons - this is based on subjective seasonal patterns. We will define 3 seasons: cool = Dec-Feb; hot = Jun-Sept; average = Mar-May, Oct-Nov.

In [ ]:
def create_seasons(df, input_col, output_col, month_bins, season_labels):
    """create seasons based on predefined season labels"""
    df[output_col] = pd.cut(df[input_col].astype(str).str[5:7].astype(int), bins=month_bins, labels=season_labels,
                            ordered=False)
    return df

Drop columns if the percentage of null values in the column exceeds a certain threshold

In [ ]:
def drop_null_value_columns_threshold(data, null_value_percentage_table, threshold=1.00):
    """remove columns if more than threshold % of its values are null"""
    null_value_fields = null_value_percentage_table[(null_value_percentage_table > threshold)].index
    df = data.drop(null_value_fields, axis=1)
    return df

Drop user-defined column(s) based on column name(s)

In [ ]:
def drop_null_value_columns_fields(data, fields_to_drop):
    """remove columns if the column is in a specific list of fields"""
    df = data.drop(fields_to_drop, axis=1)
    return df

Perform cube root transformation. Based on some previous histogram plotting in our sandbox trial and error testing, the curr_usage field was skewed. Transforming this field by taking its cube root normalizes its data:

In [ ]:
def cube_rt_transform(data, input_col, output_col):
    """create a cube root transformation output column from a certain input column"""
    data[output_col] = np.cbrt(data[input_col])
    return data

Bin columns from many numerical values to a few higher-level categorized levels. This was intended for the home_value and term_length columns, as these had very granular data, and for home_value was heavily imputed so those values themselves may not have meant as much as the strata of that they are in conveyed:

In [ ]:
def binning_transform(data, input_col, output_col, bin_cutoffs, bin_labels):
    """create a cube root transformation column an the output of a defined input column"""
    data[output_col] = pd.cut(data[input_col], bins=bin_cutoffs, labels=bin_labels)
    return data

### Data Cleaning + Feature Engineering Pipeline

When running this final pipeline, we only use the relevant transformations. We removed transformations that were not useful. The sandbox file has the commented out versions of other transformations that were not useful.

Here we define the final parameters that will be used in the data cleaning pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

drop_cols = ['order_day', 'customer_id']
month_bins = [0, 2, 5, 9, 11, 12]
season_labels = ['cool', 'average', 'hot', 'average', 'cool']
null_value_percentage_tbl = product_x_train.isnull().sum() / product_x_train.shape[0]
term_length_bins = [0, 12, 24, 36, 48, 60, 99999999]
term_length_bin_labels = ['up_to_1_year', '1-2_years', '2-3_years', '3-4_years', '4-5_years', '5+_years']
data_type_transformations = {'order_day': 'datetime64[ns]', 'tos_flg': object, 'disconotice_flg': bool,
                             'oam_activelogin_cnt': np.int64, 'term_length': object, 'called_numcalls_cnt': np.int64,
                             'latefee_flg': bool, 'dwelling_type_cd': object, 'curr_usage': np.float64,
                             'product_type_cd': object, 'pool': object, 'automatic_payment_flg': bool,
                             'weblog_flg': bool, 'deposit_onhand_amt': np.float64, 'ebill_enroll_flag': bool,
                             'called_flg': bool, 'oam_flg': bool, 'sap_productname': object, 'numweblog_cnt': np.int64,
                             'disconnects_flg': bool, 'load_profile': object, 'city': object, 'zipcode': object,
                             'home_value': np.float64, 'county': object, 'tdsp': object, 'dma': object,
                             'ev_driver': bool, 'segment': np.int64, 'customer_id': object, 'meter_id': object}

Here we take the above parameters and apply them to the data via the pipeline

In [ ]:
#run the pipeline - in this final version, only use the relevant values, cleared out irrelevant transformations
data_cleaning_pipeline = Pipeline(steps=[
    ("convert_dtypes",
     FunctionTransformer(convert_dtypes, kw_args={"data_type_transformation": data_type_transformations})),
    ("replace_NAs", FunctionTransformer(replace_nan, kw_args={"col": 'tos_flg'})),
    ("replace_blanks", FunctionTransformer(replace_blank, kw_args={"col": 'deposit_onhand_amt'})),
    ("left_strip", FunctionTransformer(left_strip, kw_args={"col": 'zipcode', "num_char": 5})),
    ("replace_boolean", FunctionTransformer(replace_boolean)),
    ("replace_yn", FunctionTransformer(replace_yn)),
    ("replace_non_numeric", FunctionTransformer(replace_non_numeric, kw_args={"col": 'term_length'})),
    ("create_seasons", FunctionTransformer(create_seasons, kw_args={"input_col": 'order_day', "output_col": 'seasons',
                                                                    "month_bins": month_bins,
                                                                    "season_labels": season_labels})),
    ("drop_null_value_columns_thresh", FunctionTransformer(drop_null_value_columns_threshold, kw_args={
        "null_value_percentage_table": null_value_percentage_tbl, "threshold": 0.50})),
    ("drop_null_value_columns_dropcols",
     FunctionTransformer(drop_null_value_columns_fields, kw_args={"fields_to_drop": drop_cols})),
    ("cube_rt_transform",
     FunctionTransformer(cube_rt_transform, kw_args={"input_col": 'curr_usage', "output_col": "curr_usage_cbrt"})),
    ("binning_transform_term_length", FunctionTransformer(binning_transform, kw_args={'input_col': 'term_length',
                                                                                      'output_col': 'term_length_binned',
                                                                                      'bin_cutoffs': term_length_bins,
                                                                                      'bin_labels': term_length_bin_labels}))
])

### Imputation and Scaling Pipeline

Here we impute and scale the numeric data and impute and one hot encode the categorical data. These final imputation, scaling, and OHE strategies are based on previous experiments:

In [ ]:
#create a feature engineering pipeline to impute and scale data
from sklearn.compose import make_column_selector
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

feature_engineering_col_trans = ColumnTransformer(transformers=[
    ('numeric', Pipeline([
        ('impute_mean', SimpleImputer(strategy="mean")),
        ('scaler_minmax', MinMaxScaler())
    ]), make_column_selector(dtype_include=np.number)),
    ('categorical', Pipeline([
        ('impute_mode', SimpleImputer(strategy="most_frequent")),
        ('OHE_500', OneHotEncoder(min_frequency=500, handle_unknown="ignore"))
    ]), make_column_selector(dtype_include=object))
])

We combine the above data cleaning, feature engineering, imputation, scaling pipelines into 1 combined preprocessing pipeline

In [ ]:
preprocessing_pipeline = Pipeline([
    ('data_cleaning_pipeline', data_cleaning_pipeline),
    ('feature_engineering_pipeline', feature_engineering_col_trans)])

## Data Modeling

### Modeling Functions

#### Model Inclusion

We define a function to combine the above preprocessing pipeline and append a modeling step to the end of it

In [ ]:
def model_pipeline_creation(preprocessing_pipe, model):
    '''combine the above preprocessing pipeline and append a modeling step to the end of it'''
    return Pipeline(steps=[
        ('preprocessing', preprocessing_pipe),
        ('model', model)
    ])

#### Hyperparameter Tuning

We define a function that performs hyperparameter tuning via randomized search to maximize the ROC AUC

In [ ]:
def hyperparameter_tuning(model_pipe, hyperparameter_search_space, iterations, cv_folds, scoring_rules):
    '''performs hyperparameter tuning via randomized search to maximize the ROC AUC'''
    return RandomizedSearchCV(model_pipe, param_distributions=hyperparameter_search_space, n_iter=iterations,
                              cv=cv_folds,
                              scoring=scoring_rules, random_state=0, refit='roc_auc')

#### Model Refitting

We define a function that takes the tuned-hyperparameter model and re-fits the train data on it

In [ ]:
def model_fitting(pipe_hyper_tuned, x_train_data, y_train_data):
    '''takes the tuned-hyperparameter model and re-fits the train data on it'''
    return pipe_hyper_tuned.fit(x_train_data, y_train_data)

#### Confusion Matrix Creation

We define a function that takes the tuned model and predicts it using the test data, and then creates the confusion matrix from that test data predictions

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

plt.rcParams['axes.grid'] = False


def create_confusion_matrix(pipe_hyper_tuned, x_test_data, y_test_data):
    '''takes the tuned model and predicts it using the test data, and then creates the confusion matrix from that test data'''
    model_cm = confusion_matrix(y_test_data, pipe_hyper_tuned.predict(x_test_data), labels=pipe_hyper_tuned.classes_,
                                normalize='all')
    model_cm_disp = ConfusionMatrixDisplay(confusion_matrix=model_cm, display_labels=pipe_hyper_tuned.classes_)
    model_cm_disp.plot(cmap='Blues')

#### Plot ROC AUC curve

We define a function that takes the tuned model and creates the ROC AUC curve from those test data predictions

In [ ]:
from sklearn.metrics import RocCurveDisplay


def plot_roc_auc_curve(pipe_hyper_tuned, x_test_data, y_test_data):
    '''takes the tuned model and creates the ROC AUC curve from those test data'''
    return RocCurveDisplay.from_estimator(pipe_hyper_tuned, x_test_data, y_test_data)

#### Model Scoring

We define a function that takes the optimized model and provides its best AUC score on the train and test data, the best hyperparameters, and the best F2 score from that test data predictions, as well as a detailed dataframe showing the model performance

In [ ]:
def model_scoring(pipe_hyper_tuned, x_test_data, y_test_data):
    '''takes the optimized model and provides its best AUC score on the train and test data, the best hyperparameters, and the best F2 score from that test data predictions, as well as a detailed dataframe showing the model performance'''
    print("Best ROC AUC Score of train set: " + str(pipe_hyper_tuned.best_score_))
    print("Best parameter set: " + str(pipe_hyper_tuned.best_params_))
    print("Test ROC AUC Score: " + str(pipe_hyper_tuned.score(x_test_data, y_test_data)))
    print("Test F2 Score: " + str(fbeta_score(y_test_data, pipe_hyper_tuned.predict(x_test_data), beta=2)))
    return pd.DataFrame(pipe_hyper_tuned.cv_results_).sort_values(by='rank_test_roc_auc')

#### Full model fitting and scoring

We define a function that takes all the above modeling pipeline steps, and puts it together in one consolidated pipeline

In [ ]:
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer
from scipy.stats import randint


def modeling_pipeline_full(preprocessing_pipe, model_name, model_param_dist, iterations, cv_folds, scoring_rules,
                           x_train_data, y_train_data, x_test_data, y_test_data):
    '''takes all the above modeling pipeline steps, and puts it together in one consolidated pipeline'''
    model_pipeline_function = model_pipeline_creation(preprocessing_pipe, model_name)
    model_pipeline_hyperparameter_tuned = hyperparameter_tuning(model_pipeline_function, model_param_dist, iterations,
                                                                cv_folds, scoring_rules)
    model_model_train = model_fitting(model_pipeline_hyperparameter_tuned, x_train_data, y_train_data)
    create_confusion_matrix(model_pipeline_hyperparameter_tuned, x_test_data, y_test_data)
    plot_roc_auc_curve(model_pipeline_hyperparameter_tuned, product_x_test, product_y_test)
    return model_scoring(model_pipeline_hyperparameter_tuned, x_test_data, y_test_data)

### Modeling Experiments

Define the scoring metrics we'll calculate for each model. We will score each model on its ROC AUC and its F2 score

In [ ]:
scoring = {"roc_auc": 'roc_auc', 'f2_scorer': make_scorer(fbeta_score, beta=2)}

#### Logistic Regression

Run model through the full preprocessing/modeling pipeline, using the defined hyperparameter search space and 2 iterations of 5-fold cross-validation. This will output the model scoring as well as the confusion matrix and AUC plot.

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(random_state=0)
log_reg_param_dist = {'model__C': randint(low=1, high=10),
                      'model__class_weight': [None, 'balanced']}

modeling_pipeline_full(preprocessing_pipeline, log_reg, log_reg_param_dist, 2, 5, scoring, product_x_train,
                       product_y_train, product_x_test, product_y_test)

#### KNN

Run model through the full preprocessing/modeling pipeline, using the defined hyperparameter search space and 2 iterations of 5-fold cross-validation. This will output the model scoring as well as the confusion matrix and AUC plot.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

knn_param_dist = {'model__n_neighbors': randint(low=2, high=10)}

modeling_pipeline_full(preprocessing_pipeline, knn, knn_param_dist, 2, 5, scoring, product_x_train, product_y_train,
                       product_x_test, product_y_test)

#### SGD Classifier

Run model through the full preprocessing/modeling pipeline, using the defined hyperparameter search space and 2 iterations of 5-fold cross-validation. This will output the model scoring as well as the confusion matrix and AUC plot.

In [ ]:
from sklearn.linear_model import SGDClassifier
import random

sgdc = SGDClassifier(max_iter=10, tol=1e-1, random_state=0)

sgdc_param_dist = {'model__alpha': (random.random() / 10000, random.random() / 1000),
                   'model__class_weight': [None, 'balanced']}

modeling_pipeline_full(preprocessing_pipeline, sgdc, sgdc_param_dist, 2, 5, scoring, product_x_train, product_y_train,
                       product_x_test, product_y_test)


#### Decision Tree

Run model through the full preprocessing/modeling pipeline, using the defined hyperparameter search space and 2 iterations of 5-fold cross-validation. This will output the model scoring as well as the confusion matrix and AUC plot.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier(random_state=0)

dtree_param_dist = {'model__max_features': randint(low=10, high=100),
                    'model__max_depth': randint(low=2, high=20),
                    'model__class_weight': [None, 'balanced']}

modeling_pipeline_full(preprocessing_pipeline, dtree, dtree_param_dist, 2, 5, scoring, product_x_train,
                       product_y_train, product_x_test, product_y_test)


#### Random Forest

Run model through the full preprocessing/modeling pipeline, using the defined hyperparameter search space and 2 iterations of 5-fold cross-validation. This will output the model scoring as well as the confusion matrix and AUC plot.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0)

rf_param_dist = {'model__n_estimators': randint(low=50, high=500),  #165
                 'model__max_features': randint(low=10, high=50),  #30
                 'model__max_depth': randint(low=2, high=20),  #11
                 'model__class_weight': [None, 'balanced']}  #balanced

modeling_pipeline_full(preprocessing_pipeline, rf, rf_param_dist, 2, 5, scoring, product_x_train, product_y_train,
                       product_x_test, product_y_test)


#### ExtraTrees

Run model through the full preprocessing/modeling pipeline, using the defined hyperparameter search space and 2 iterations of 5-fold cross-validation. This will output the model scoring as well as the confusion matrix and AUC plot.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

xtree = ExtraTreesClassifier(random_state=0)

xtree_param_dist = {'model__n_estimators': randint(low=50, high=500),
                    'model__max_features': randint(low=5, high=50),
                    'model__max_depth': randint(low=2, high=20),
                    'model__class_weight': [None, 'balanced']}

modeling_pipeline_full(preprocessing_pipeline, xtree, xtree_param_dist, 2, 5, scoring, product_x_train,
                       product_y_train, product_x_test, product_y_test)

#### LightGBM

Run model through the full preprocessing/modeling pipeline, using the defined hyperparameter search space and 2 iterations of 5-fold cross-validation. This will output the model scoring as well as the confusion matrix and AUC plot.

In [ ]:
#https://www.kaggle.com/code/prashant111/lightgbm-classifier-in-python
import lightgbm as lgbm

lightgbm = lgbm.LGBMClassifier(random_state=0)

lightgbm_param_dist = {'model__max_depth': randint(low=1, high=10),  #6, 4
                       'model__num_leaves': randint(low=50, high=250),  #117, 59, 137
                       'model__n_estimators': randint(low=50, high=250),  #245, 86
                       'model__class_weight': [None, 'balanced']}  #balanced

modeling_pipeline_full(preprocessing_pipeline, lightgbm, lightgbm_param_dist, 2, 5, scoring, product_x_train,
                       product_y_train, product_x_test, product_y_test)


#### Voting Classifier

Run model through the full preprocessing/modeling pipeline, using the defined hyperparameter search space and 2 iterations of 5-fold cross-validation. This will output the model scoring as well as the confusion matrix and AUC plot.

In [ ]:
from sklearn.ensemble import VotingClassifier

voting = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=0)),
        ('rf', RandomForestClassifier(random_state=0)),
        ('lgbm', lgbm.LGBMClassifier(random_state=0))
    ], voting='soft')

voting_param_dist = {'model__lr__C': randint(low=1, high=10),  #2,3 optimal
                     'model__rf__n_estimators': randint(low=50, high=500),  #387
                     'model__rf__max_features': randint(low=50, high=200),  #98, 97 optimal
                     'model__rf__max_depth': randint(low=2, high=20),  #16,6 optimal
                     'model__lgbm__max_depth': randint(low=5, high=20),  #13,12 optimal
                     'model__lgbm__num_leaves': randint(low=50, high=250),  #190, 71 optimal
                     'model__lgbm__n_estimators': randint(low=50, high=250),  #138
                     'model__lr__class_weight': [None, 'balanced'],  #none
                     'model__rf__class_weight': [None, 'balanced'],  #balanced
                     'model__lgbm__class_weight': [None, 'balanced']}  #none

modeling_pipeline_full(preprocessing_pipeline, voting, voting_param_dist, 2, 5, scoring, product_x_train,
                       product_y_train, product_x_test, product_y_test)

#### Stacking Classifier

Run model through the full preprocessing/modeling pipeline, using the defined hyperparameter search space and 2 iterations of 5-fold cross-validation. This will output the model scoring as well as the confusion matrix and AUC plot.

In [ ]:
from sklearn.ensemble import StackingClassifier

stacking = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=0)),
        ('rf', RandomForestClassifier(random_state=0)),
        ('lgbm', lgbm.LGBMClassifier(random_state=0))
    ],
    final_estimator=LogisticRegression(random_state=0),
    cv=2  # number of cross-validation folds
)

stacking_param_dist = {'model__lr__C': randint(low=1, high=10),  #4
                       'model__rf__n_estimators': randint(low=50, high=500),  #120
                       'model__rf__max_features': randint(low=10, high=100),  #19
                       'model__rf__max_depth': randint(low=2, high=20),  #9
                       'model__lgbm__max_depth': randint(low=1, high=10),  #6
                       'model__lgbm__num_leaves': randint(low=50, high=250),  #117
                       'model__lgbm__n_estimators': randint(low=50, high=350),
                       'model__lr__class_weight': [None, 'balanced'],
                       'model__rf__class_weight': [None, 'balanced'],
                       'model__lgbm__class_weight': [None, 'balanced']}

modeling_pipeline_full(preprocessing_pipeline, stacking, stacking_param_dist, 2, 5, scoring, product_x_train,
                       product_y_train, product_x_test, product_y_test)


#### Neural Network - MLP Classifier

Run model through the full preprocessing/modeling pipeline, using the defined hyperparameter search space and 2 iterations of 5-fold cross-validation. This will output the model scoring as well as the confusion matrix and AUC plot.

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=50, random_state=0)

mlp_param_dist = {'model__hidden_layer_sizes': [(50, 50, 50), (50, 100, 50), (100,), (10, 30, 10), (20,)],
                  'model__activation': ['tanh'],  #tanh is consistently better than relu
                  'model__alpha': [0.0001, 0.05],
                  'model__learning_rate': ['constant', 'adaptive']}

modeling_pipeline_full(preprocessing_pipeline, mlp, mlp_param_dist, 2, 5, scoring, product_x_train, product_y_train,
                       product_x_test, product_y_test)

## True Test Data Probability Predictions

Predict probabilities using winning model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

#create the test data from the to be provided true test data
product_true_test_x_data = pd.read_csv('product_sales_test.csv')


def modeling_pipeline_test_predictions(preprocessing_pipe, model_name, model_param_dist, iterations, cv_folds,
                                       scoring_rules, x_train_data, y_train_data, x_test_data_true):
    '''re-run the preprocessing/modeling pipeline, but predict the model probabilities'''
    model_pipeline_function = model_pipeline_creation(preprocessing_pipe, model_name)
    model_pipeline_hyperparameter_tuned = hyperparameter_tuning(model_pipeline_function, model_param_dist, iterations,
                                                                cv_folds, scoring_rules)
    model_model_train = model_fitting(model_pipeline_hyperparameter_tuned, x_train_data, y_train_data)
    model_model_predictions = model_pipeline_hyperparameter_tuned.predict_proba(x_test_data_true)
    return model_model_predictions


Given the above updated function, apply it to the random forest model

In [ ]:
scoring = {"roc_auc": 'roc_auc', 'f2_scorer': make_scorer(fbeta_score, beta=2)}

rf = RandomForestClassifier(random_state=0)

rf_param_dist = {'model__n_estimators': randint(low=50, high=500),  #165
                 'model__max_features': randint(low=10, high=50),  #30
                 'model__max_depth': randint(low=2, high=20),  #11
                 'model__class_weight': [None, 'balanced']}  #balanced

#create the model predictions for the best model (RF), 3 iterations of 5-fold CV
best_model_predictions = modeling_pipeline_test_predictions(preprocessing_pipeline, rf, rf_param_dist, 2, 5,
                                                            scoring, product_x_train, product_y_train,
                                                            product_true_test_x_data)


Check to make sure number of test rows == the number of prediction rows (yes, both are 20,328 rows)

In [ ]:
assert (product_true_test_x_data.shape[0] == len(best_model_predictions[:, 1]))

We check this model's predictions. The left column is probability of a row being classified as 0, right column is probability of classified as 1. To get the probabilities of acceptance we take just column 1. These are the final prediction probabilities we will output to csv and send:

In [ ]:
best_model_predictions

We plot the distribution of acceptance prediction probabilities. As expected, most probabilities are closer to 0:

In [ ]:
plt.hist(best_model_predictions[:, 1])

We also plot the distribution of the best model predictions on the true test set (converted to 0 and 1) and it is a large majority 0's, as expected:

In [ ]:
plt.hist(np.argmax(best_model_predictions, 1))